# This is a time_sync helper notebook for annotating timestamps

## Video Annotation
1. **Note** through quickTime Player the "video_start_time_SMPTE" (last 1 or 2 digits -> x frames)
2. Find the number of frames used in the video (30 or 60)
3. Translate frames to milliseconds with x/fps (will give the ms approximation)
4. **Note** "video_start_time"  
5. **Annotate** "relative_video_clap_time" in relative seconds since start
6. **Note** "video_clap_time" by calculating the real time (video start time + seconds elapsed)

## IMU Annotation

In [ ]:
%matplotlib notebook
import json
import pickle
from datetime import datetime, timedelta
import pandas as pd

from src.main.imu_data_io import load_raw_sensor_data
from src.main.imu_preprocessing import (sync, resample)

from src.utils.imu_data_visualization import plot_raw_sensor

In [ ]:
with open('../config/imu_config.json') as config_file:
    config = json.load(config_file)

In [ ]:
raw_data_directory = config['data_paths']['raw_data_directory']
processed_data_directory = config['data_paths']['processed_data_directory']
upsample_frequency = config['resampling']['upsample_frequency']
filter_length = config['filtering']['moving_average_filter_length']
gravity_filter_cutoff_hz = config['filtering']['gravity_filter_cutoff_hz']
left_handed_subjects = config['processing_options']['left_handed_subjects']
saving_format = config['saving_options']['file_format']
path_to_mean_std = config['standardization']['path_to_mean_std']

In [ ]:
subject_id = "19"

In [ ]:
path = f"../data/raw/{subject_id}"

In [ ]:
acc_data, gyro_data = load_raw_sensor_data(path)

In [ ]:
sync_acc, sync_gyro = sync(acc_data, gyro_data)
res_acc, res_gyro = resample(acc_data, gyro_data, upsample_frequency)

1. **Note** "imu_start_time"
2. Identify event (clap) from resampled data
3. **Note** "imu_clap_time"

In [ ]:
plot_raw_sensor(res_acc, "")

1. FIND DT between imu and video
2. **Note** "video_imu_time_difference"
3. **Note** "video_imu_time_difference_seconds"

In [ ]:
# IMU actual clap time
imu_clap_time_str = '15:36:41.3020'  # hh:mm:ss.ms format
imu_clap_time = datetime.strptime(imu_clap_time_str, '%H:%M:%S.%f')

# Video actual clap time
video_clap_time_str = '16:37:14.5630'  # hh:mm:ss.ms format
video_clap_time = datetime.strptime(video_clap_time_str, '%H:%M:%S.%f')

# Calculate dt (time difference)
dt = video_clap_time - imu_clap_time

# Output dt
print("Time Difference (dt):", dt)

dt_seconds = dt.total_seconds()

print("Time Difference in seconds:", dt_seconds)

# OPTIONAL

## Mandometer Annotation

1. Find commonly identifiable event
2. **Note** "relative_video_mando_time" which is the relative seconds in *video* time of when it happened
3. **Note** "relative_mando_time" which is the relative seconds in *mandometer* time of when it happened

In [ ]:
with open("time_sync.json", "r") as f:
    time_sync = json.load(f)

In [ ]:
sync_subject = [info for info in time_sync['sync_info'] if info['subject_id'] == subject_id][0]

In [ ]:
video_start_time_str = sync_subject['video_start_time']
relative_video_mando_time_str = sync_subject['relative_video_mando_time']  # Find the relative time since the video started of a commonly identifiable event
relative_mando_time_str = sync_subject['relative_mando_time'] # the sample number of when the person started eating. since median sample rate is 1hz ->seconds (beware cause average hz is less than that: 0.94hz)

1. Find when the mandometer actually started in video's timeline
2. **Note** "mando_video_start_time"

In [ ]:
# Convert string times to datetime objects
video_start_time = datetime.strptime(video_start_time_str, "%H:%M:%S.%f")
relative_video_mando_time = datetime.strptime(relative_video_mando_time_str, "%H:%M:%S.%f")
relative_mando_time = datetime.strptime(relative_mando_time_str, "%H:%M:%S.%f")

In [ ]:
# Calculate timedelta for the relative times from the video start to the event (n) and from the device start to the event (k)
relative_video_mando_delta = timedelta(hours=relative_video_mando_time.hour, minutes=relative_video_mando_time.minute, seconds=relative_video_mando_time.second, microseconds=relative_video_mando_time.microsecond)
relative_mando_delta = timedelta(hours=relative_mando_time.hour, minutes=relative_mando_time.minute, seconds=relative_mando_time.second, microseconds=relative_mando_time.microsecond)

# Calculate the device's start time using the logic X + n - k
mando_video_start_time = (video_start_time + relative_video_mando_delta - relative_mando_delta).strftime("%H:%M:%S.%f")


In [ ]:
print("Mandometer's approximate start time:", mando_video_start_time)